In [ ]:
# 对骨干网络的测试
import torch
from models.backbone import ResNet, Conv_4

input = torch.randn(32, 3, 500, 333).cuda()  # (batch_size, channels, w, h)
resnet = ResNet.resnet12().cuda()
conv = Conv_4.BackBone(64).cuda()

output_resnet = resnet(input)
output_conv = conv(input)

print(output_resnet.shape)
print(output_conv.shape)

del input
del resnet
del conv
del output_resnet
del output_conv

In [19]:
torch.cuda.empty_cache()
del input
del resnet
del conv
del output_resnet
del output_conv

In [34]:
# 对roi_align的测试
import torch
from torchvision.ops import roi_align

input = torch.randn(32, 1, 4, 4)
boxes = torch.tensor([[0, 0, 0, 4, 4]]).float()  # [batch_id, x1, y1, x2, y2]
output_size = [4, 4]
output_align_false = roi_align(input=input, boxes=boxes, output_size=output_size, aligned=False)  # 如果正好对齐, 对周围点进行采样
output_align_true = roi_align(input=input, boxes=boxes, output_size=output_size, aligned=True)  # 如果正好对齐, 不对周围点进行采样
input_box = input[0, :, 0:4, 0:4].unsqueeze(0)
print(input_box.shape)
print(output_align_false.shape)
print(output_align_true.shape)
print(input_box)
print(output_align_false)
print(output_align_true)

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 4, 4])
tensor([[[[ 1.3304,  0.6126, -0.2445, -2.0494],
          [ 0.3161,  1.0369, -0.4708,  1.2764],
          [-0.3150,  1.5574,  1.0487, -0.3043],
          [-0.5418, -0.7350, -0.3038, -0.1649]]]])
tensor([[[[ 0.8240,  0.2336, -0.3721, -0.3865],
          [ 0.6488,  0.7931,  0.3875,  0.4861],
          [-0.0086,  0.3918,  0.0689, -0.2346],
          [-0.6384, -0.5194, -0.2344, -0.1649]]]])
tensor([[[[ 1.3304,  0.6126, -0.2445, -2.0494],
          [ 0.3161,  1.0369, -0.4708,  1.2764],
          [-0.3150,  1.5574,  1.0487, -0.3043],
          [-0.5418, -0.7350, -0.3038, -0.1649]]]])


In [60]:
# 对roi_align的测试
import torch
from torchvision.ops import roi_align

input = torch.randn(1, 1, 4, 4)
boxes = torch.tensor([[0, 0, 0, 4, 4], [0, 0, 0, 3, 4], [0, 0, 0, 4, 3]]).float()  # [batch_id, x1, y1, x2, y2]
output_size = [4, 4]
output_align_false = roi_align(input=input, boxes=boxes, output_size=output_size, aligned=False)  # 如果正好对齐, 对周围点进行采样
output_align_true = roi_align(input=input, boxes=boxes, output_size=output_size, aligned=True)  # 如果正好对齐, 不对周围点进行采样
input_box = input[0, :, 0:4, 0:4].unsqueeze(0)
print(input_box.shape)
print(output_align_false.shape)
print(output_align_true.shape)
print(input_box)
print(output_align_false)
print(output_align_true)

torch.Size([1, 1, 4, 4])
torch.Size([3, 1, 4, 4])
torch.Size([3, 1, 4, 4])
tensor([[[[ 0.3673, -1.2673,  0.3280,  0.0025],
          [ 0.8189,  0.1512, -1.9172, -1.5804],
          [ 0.1671,  1.1049,  0.6560, -1.6464],
          [ 0.7830, -0.7810,  1.1189, -0.6382]]]])
tensor([[[[ 1.7514e-02, -6.7634e-01, -7.9177e-01, -7.8894e-01],
          [ 5.6051e-01, -1.3060e-03, -1.1220e+00, -1.6134e+00],
          [ 3.1846e-01,  5.2469e-01, -1.2741e-01, -1.1423e+00],
          [ 9.6187e-04,  1.6895e-01,  2.4036e-01, -6.3821e-01]]],


        [[[ 1.6141e-01, -5.8764e-01, -7.6504e-01, -7.9106e-01],
          [ 5.4363e-01,  4.7069e-01, -4.7330e-01, -1.2449e+00],
          [ 3.5760e-01,  2.5260e-01,  7.9677e-01, -3.8113e-01],
          [ 1.9646e-01, -5.4353e-01,  8.8144e-01,  2.0720e-02]]],


        [[[-9.9373e-02, -6.2466e-01, -5.5251e-01, -5.9107e-01],
          [ 5.0393e-01, -6.6260e-01, -1.5921e+00, -1.5887e+00],
          [ 6.1710e-01,  6.5999e-01, -6.5189e-01, -1.6381e+00],
          [ 2.3909

In [37]:
# 对rpn的测试
import torch
from models.region_proposal_network.rpn import RegionProposalNetwork

rpn = RegionProposalNetwork(64, 64,
                            ratios=[0.5, 1, 2],
                            anchor_scales=[8, 16, 32],
                            feat_stride=16,
                            mode="training")
input = torch.randn(32, 64, 128, 128)
img_size = input.shape[2:]
print(img_size)
rpn_locs, rpn_scores, rois, roi_indices, anchor = rpn(x=input, img_size=img_size)
print('rpn_locs:    ', rpn_locs.shape)
print('rpn_scores:  ', rpn_scores.shape)
print('rois:        ', rois.shape)
print('roi_indices: ', roi_indices.shape)
print('anchor:      ', anchor.shape)
# rpn_locs：rpn对位置的修正，大小[1, 16650, 4]
# rpn_scores ：rpn判断区域前景背景，大小[1, 16650, 2]
# rois：rpn筛选出的roi的位置，大小[300， 4]
# roi_indices：rpn筛选出的roi对应的图片索引，大小[300]
# anchor：原图像的锚点，大小[16650, 4]
# 其中，16650是放缩后的图像所产生的所有锚点（37*50*9），每个锚点都对应了一个rp。通过 rpn_scores以及nms可以得到筛选后的大小为300的roi。

torch.Size([128, 128])
rpn_locs:     torch.Size([32, 147456, 4])
rpn_scores:   torch.Size([32, 147456, 2])
rois:         torch.Size([32, 600, 4])
roi_indices:  torch.Size([32, 600])
anchor:       torch.Size([1, 147456, 4])


In [1]:
# 对模型的测试
import torch
import random
from models.FRNOD import FRNOD
import numpy as np
from models.feature_reconstruction_net.FRN import FRN

way = 5
shot = 2
support = torch.randn([5, 3, 128, 128]).cuda()
query = torch.randn([3, 3, 128, 128]).cuda()

bboxs = torch.empty(0)
n = 10  # n为图像张数
for i in range(n):  # i为图像的index
    # print('------------------------')
    support_shot = 5  # 框数
    bbox = torch.randint(low=0, high=128, size=(1, support_shot, 4)).float()  # 对图像生成k个框
    # print(bbox.shape)
    bboxs = torch.cat([bboxs, bbox], 0)

print('bboxs.shape', bboxs.shape)
boxes_np = bboxs.numpy()

labels = [random.randint(0, way) for i in range(bboxs.shape[0])]

# print(bboxs)

model = FRNOD(way=10, shot=114, num_categories=3).cuda()
model.forward_train(support_imgs=support, query_imgs=query, bboxes_gt=boxes_np, labels=labels)

bboxs.shape torch.Size([10, 5, 4])
torch.Size([3, 3, 128, 128])
torch.Size([3, 640, 8, 8])
(576, 4) (5, 4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(605, 4) (5, 4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


G:\Code\FRNOD\models\FRNOD.py:504: RuntimeWarning: invalid value encountered in log
  dw = np.log(base_width / width)


TypeError: 'int' object is not subscriptable

In [ ]:
# 数据集读取测试
from torchvision.datasets.coco import CocoDetection
from torchvision.transforms import transforms

# root : coco图片路径
# annFile : 标注文件路径
# transform : 图像转换(用于PIL)
# target_transform : 标注转换
# transforms : 图像和标注的转换
cococD = CocoDetection(root='datasets/fsod/images', annFile='datasets/fsod/annotations/fsod_train.json',
                       transform=transforms.ToTensor())

In [3]:
import cv2


def xywh2xyxy(xywh: list):
    x, y, w, h = xywh
    xyxy = [x, y, x + w, y + h]
    return xyxy


font = cv2.FONT_HERSHEY_SIMPLEX
for i in range(len(cococD)):
    bboxes = []
    ids = []
    img, labels = cococD.__getitem__(i)

    for label in labels:
        print(label['bbox'])
        bboxes.append(xywh2xyxy(label['bbox']))
        ids.append(label['category_id'])

    img = img.permute(1, 2, 0).numpy()
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    for box, id_ in zip(bboxes, ids):
        x1 = int(box[0])
        y1 = int(box[1])
        x2 = int(box[2])
        y2 = int(box[3])
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), thickness=5)
        cv2.putText(img, text=str(id_), org=(x1 + 5, y1 + 5), fontFace=font, fontScale=1,
                    thickness=2, lineType=cv2.LINE_AA, color=(0, 255, 0))

    cv2.imshow('test', img)
    if ord('q') == cv2.waitKey(0):
        break
cv2.destroyAllWindows()

NameError: name 'cococD' is not defined

In [2]:
from utils.dataset_tools.coco import read_coco_detection, show_dataset

fsod_dataset = read_coco_detection()
show_dataset(fsod_dataset, show_anns=True)

loading annotations into memory...
Done (t=1.57s)
creating index...
index created!


In [4]:
print(cococD)

image, target = cococD.__getitem__(0)
print(image)
# <PIL.Image.Image image mode=RGB size=1024x719 at 0x200CCD0F8C8>
print(target)
# [{'ignore': 0, 'image_id': 2019000000000, 'segmentation': 0, 'bbox': [393.0, 347.0, 48.0, 42.0], 'area': 0.0, 'category_id': 1, 'iscrowd': 0, 'id': 0}]
# box: x, y, w, h
# COCO数据集格式中，bbox 的保存格式为 [x, y, w, h]
# 如果需要转换为[x1,y1,x2,y2]，可以通过如下进行转换
# bbox = [x1, y1, x1 + w - 1, y1 + h - 1]

Dataset CocoDetection
    Number of datapoints: 52350
    Root location: datasets/fsod/images
    StandardTransform
Transform: ToTensor()
tensor([[[0.8667, 0.8745, 0.8863,  ..., 0.4941, 0.4902, 0.4863],
         [0.8667, 0.8667, 0.8745,  ..., 0.4941, 0.4902, 0.4863],
         [0.8706, 0.8667, 0.8667,  ..., 0.4902, 0.4902, 0.4863],
         ...,
         [0.8863, 0.8863, 0.8824,  ..., 0.5216, 0.5255, 0.5255],
         [0.8980, 0.8902, 0.8824,  ..., 0.5294, 0.5255, 0.5255],
         [0.8980, 0.8902, 0.8863,  ..., 0.5294, 0.5255, 0.5255]],

        [[0.8549, 0.8627, 0.8745,  ..., 0.0000, 0.0039, 0.0000],
         [0.8549, 0.8549, 0.8627,  ..., 0.0000, 0.0039, 0.0000],
         [0.8588, 0.8549, 0.8549,  ..., 0.0039, 0.0039, 0.0000],
         ...,
         [0.8667, 0.8667, 0.8627,  ..., 0.0000, 0.0000, 0.0000],
         [0.8784, 0.8706, 0.8627,  ..., 0.0039, 0.0000, 0.0000],
         [0.8784, 0.8706, 0.8667,  ..., 0.0039, 0.0000, 0.0000]],

        [[0.7333, 0.7412, 0.7529,  ..., 0.0118, 0.

In [ ]:
from torchvision.models.detection.rpn import RegionProposalNetwork, AnchorGenerator, RPNHead
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.detection.roi_heads import roi_align
from torch.utils.data.dataloader import DataLoader
from models.backbone.ResNet import resnet12
from torchvision.models.detection.image_list import ImageList
from utils.dataset_tools.coco import read_coco_detection, show_dataset

# root : coco图片路径
# annFile : 标注文件路径
# transform : 图像转换(用于PIL)
# target_transform : 标注转换
# transforms : 图像和标注的转换

fsod_dataset = read_coco_detection()
show_dataset(fsod_dataset, show_anns=True)

DataLoader(fsod_dataset, )

In [1]:

backbone = resnet12()
anchor_generator = AnchorGenerator(sizes=((32, 64, 128)), aspect_ratios=((0.5, 1.0, 2.0)))
# head (nn.Module): module that computes the objectness and regression deltas
rpn = RegionProposalNetwork(anchor_generator=anchor_generator, head=RPNHead(in_channels=64, num_anchors=9),
                            fg_iou_thresh=0.7,
                            bg_iou_thresh=0.3, batch_size_per_image=256, positive_fraction=128, pre_nms_top_n=300,
                            post_nms_top_n=50, nms_thresh=0.8)
rpn.forward()

SyntaxError: invalid syntax (852278810.py, line 1)

In [4]:
from pycocotools import coco

fsod = coco.COCO('datasets/fsod/annotations/fsod_train.json')
ids = fsod.getCatIds()
imgIds = fsod.catToImgs[1]
print(imgIds)
img = fsod.imgs[2019000084317]
# print(fsod.imgs)
print(img)
annIds = fsod.getAnnIds(imgIds=[2019000071004], catIds=[1])
for annId in annIds:
    print(fsod.anns[annId])

loading annotations into memory...
Done (t=1.06s)
creating index...
index created!
[2019000000000, 2019000000646, 2019000000646, 2019000003474, 2019000005139, 2019000006117, 2019000006373, 2019000009200, 2019000009932, 2019000010388, 2019000012785, 2019000014384, 2019000000646, 2019000015757, 2019000014384, 2019000015969, 2019000000646, 2019000016658, 2019000012785, 2019000017438, 2019000000646, 2019000000646, 2019000019453, 2019000000646, 2019000021595, 2019000022165, 2019000022510, 2019000022570, 2019000000646, 2019000000646, 2019000024482, 2019000022165, 2019000025712, 2019000025785, 2019000000646, 2019000026955, 2019000009932, 2019000004716, 2019000033303, 2019000000646, 2019000034481, 2019000034751, 2019000035215, 2019000000646, 2019000035434, 2019000004716, 2019000000646, 2019000039274, 2019000042265, 2019000044159, 2019000000646, 2019000045271, 2019000045504, 2019000046512, 2019000046656, 2019000046736, 2019000044867, 2019000049251, 2019000000646, 2019000051515, 2019000000751, 2

In [7]:
from pycocotools import coco
import random

fsod = coco.COCO('datasets/fsod/annotations/fsod_train.json')
catIds = fsod.getCatIds()
print(catIds)
# N类
# 每个cat有k个实例
support_shot = 10  # type: int
query_shot = 5
for catId in catIds:
    imgIds = fsod.catToImgs[catId]
    sample_imgIds = random.sample(imgIds, support_shot + query_shot)

    support_imgIds = sample_imgIds[:support_shot]
    query_imgIds = sample_imgIds[support_shot:]

    support_annIds_all = fsod.getAnnIds(imgIds=support_imgIds, catIds=[catId])
    # k个实例
    support_annIds = random.sample(annIds, support_shot)

    # query的标注全部需要
    query_annIds = fsod.getAnnIds(imgIds=query_imgIds, catIds=[catId])

    for support_annId in support_annIds:
        print(fsod.loadAnns(support_annId))

loading annotations into memory...
Done (t=1.00s)
creating index...
index created!
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205,

In [1]:
from utils.data.dataset import FsodDataset

fsod = FsodDataseet(root='datasets/fsod/', annFile='datasets/fsod/annotations/fsod_train.json', support_shot=5,
                    query_shot=5, seed=114514)

support, query, ann = fsod.__getitem__(1)
print(support)
print(query)
print(ann)

loading annotations into memory...
Done (t=1.28s)
creating index...
index created!
正在为每个类别生成support和query


100%|██████████| 800/800 [01:18<00:00, 10.16it/s]

[<PIL.Image.Image image mode=RGB size=740x436 at 0x23C1E77C9C8>, <PIL.Image.Image image mode=RGB size=147x283 at 0x23C1E7884C8>, <PIL.Image.Image image mode=RGB size=135x364 at 0x23C1E7885C8>, <PIL.Image.Image image mode=RGB size=601x408 at 0x23C1E788748>, <PIL.Image.Image image mode=RGB size=131x298 at 0x23C1E788948>]
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x617 at 0x23C1E7889C8>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x23C1E788B08>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x843 at 0x23C1E788F08>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x770 at 0x23C1E78C348>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x1024 at 0x23C1E78C748>]
[[[{'ignore': 0, 'image_id': 2019000014348, 'segmentation': 0, 'bbox': [799.0, 304.0, 187.0, 180.0], 'area': 0.0, 'category_id': 2, 'iscrowd': 0, 'id': 14984}], [{'ignore': 0, 'image_id': 2019000014348, 'segmentation': 0, 'bbox': [634.0, 0.0, 282.0, 90.0], 'area

In [2]:
support, query, ann = fsod.__getitem__(1)
print(support)
print(query)
print(ann)

[<PIL.Image.Image image mode=RGB size=740x436 at 0x278B7BBB7C8>, <PIL.Image.Image image mode=RGB size=147x283 at 0x278B7BC3E88>, <PIL.Image.Image image mode=RGB size=135x364 at 0x278B7BC3EC8>, <PIL.Image.Image image mode=RGB size=601x408 at 0x278B7BC8548>, <PIL.Image.Image image mode=RGB size=131x298 at 0x278B7BC8748>]
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x617 at 0x278B7BC87C8>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x278B7BC8908>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x843 at 0x278B7BC8D08>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x770 at 0x278B7BCC148>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x1024 at 0x278B7BCC548>]
[[[{'ignore': 0, 'image_id': 2019000014348, 'segmentation': 0, 'bbox': [799.0, 304.0, 187.0, 180.0], 'area': 0.0, 'category_id': 2, 'iscrowd': 0, 'id': 14984}], [{'ignore': 0, 'image_id': 2019000014348, 'segmentation': 0, 'bbox': [634.0, 0.0, 282.0, 90.0], 'area

In [3]:
from utils.data.dataset import FsodDataset

fsod_random_test = FsodDataset(root='datasets/fsod/', annFile='datasets/fsod/annotations/fsod_train.json',
                               support_shot=5,
                               query_shot=5, seed=114514)
support, query, ann = fsod_random_test.__getitem__(1)
print(support)
print(query)
print(ann)

loading annotations into memory...
Done (t=2.72s)
creating index...
index created!
正在为每个类别生成support和query


100%|██████████| 800/800 [01:44<00:00,  7.63it/s]

[<PIL.Image.Image image mode=RGB size=740x436 at 0x2790CA4FE88>, <PIL.Image.Image image mode=RGB size=147x283 at 0x2790CA52AC8>, <PIL.Image.Image image mode=RGB size=135x364 at 0x2790CA52E88>, <PIL.Image.Image image mode=RGB size=601x408 at 0x2790CA52648>, <PIL.Image.Image image mode=RGB size=131x298 at 0x2790CA52088>]
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x617 at 0x2790CA52DC8>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024 at 0x2790CA52488>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x843 at 0x2790CA47208>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x770 at 0x2790CA449C8>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x1024 at 0x2790CA4E1C8>]
[[[{'ignore': 0, 'image_id': 2019000014348, 'segmentation': 0, 'bbox': [799.0, 304.0, 187.0, 180.0], 'area': 0.0, 'category_id': 2, 'iscrowd': 0, 'id': 14984}], [{'ignore': 0, 'image_id': 2019000014348, 'segmentation': 0, 'bbox': [634.0, 0.0, 282.0, 90.0], 'area

In [1]:
from torch.utils.data import Dataset, DataLoader
from utils.data.dataset import FsodDataset

fsod = FsodDataset(root='datasets/fsod/', annFile='datasets/fsod/annotations/fsod_train.json',
                   support_shot=5,
                   query_shot=5, seed=114514)

loading annotations into memory...
Done (t=1.26s)
creating index...
index created!
正在为每个类别生成support和query


100%|██████████| 800/800 [00:36<00:00, 21.74it/s]


In [1]:
from utils.data.dataset import FsodDataset
from utils.data.pre_process import transform_support

fsod = FsodDataset(root='datasets/fsod/', annFile='datasets/fsod/annotations/fsod_train.json',
                   support_shot=5,
                   query_shot=5, seed=114514)
support, _, _ = fsod[10]

loading annotations into memory...
Done (t=1.07s)
creating index...
index created!
正在为每个类别生成support和query


100%|██████████| 800/800 [02:52<00:00,  4.65it/s]


TypeError: empty() received an invalid combination of arguments - got (), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [4]:
del transform_support
from utils.data.pre_process import transform_support

transform_support(support)

TypeError: empty() received an invalid combination of arguments - got (), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [3]:
import json

a = {'a': 1}

with open('a.json', 'w') as f:
    json.dump(a, f)